In [1]:
%%capture
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import integrate
from scipy import interpolate
import math
import sys
from scipy import signal
import pandas as pd
import tensorflow as tf
from numba import cuda
import numba
import os
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, HTML
%run KalmanFilter.ipynb
import mpld3
mpld3.enable_notebook()


In [2]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.python.keras import backend as K 
from IPython.display import clear_output
from tensorflow.python.keras.initializers import RandomUniform
# tf.enable_eager_execution()

In [3]:
x_dim = 5
y_dim = 2
def create_input_and_output(data, just_acc=False):
    ## Needed Data
    gps = data.gps[:, 1:3]
    acc = data.acc_ERC[:, 1:3]
    vel = data.vel_ERC[:, 1:3]
    mag = data.mag[:, 1:3]
    gyro = data.gyro[:, 1:4]

    time_series = data.acc_ERC[:, 0]
#     ground_truth = data.kal_dis[:, 1:3]
    ground_truth = data.ground_truth.dis[:, 1:3]
    delta_time = np.diff(time_series, axis=0)
    delta_time = np.concatenate(([[0]], delta_time))

    # Choose which data to include in input
    if (just_acc):
        input_data = np.concatenate((gps, acc, delta_time), axis=1)
    else:
        input_data = np.concatenate((gps, acc, delta_time), axis=1)
    return input_data, ground_truth


In [4]:
def load_train_test_data(filename, train_percentage=0.8):
    #Load Data File
    data = Data_Stream(filename, load_truth=True)
    
    #Create input and ouput data
    input_data, ground_truth = create_input_and_output(data)
    
    x_dim = input_data.shape[1]
    y_dim = ground_truth.shape[1]

    num_train = int(train_percentage*len(input_data))
    num_test = len(input_data) - num_train

    training_data   = input_data[:num_train, :]
    training_labels = ground_truth[:num_train, :]

    test_data     = input_data[num_train:, :]
    test_labels   = ground_truth[num_train:, :]
    
    return training_data, training_labels, test_data, test_labels, data.gps

In [5]:
custom_scale_matrix = np.zeros((2, x_dim))
custom_scale_matrix.fill(2000)
custom_scale_matrix[1, :] = -custom_scale_matrix[1 , :]

x_scaler = MinMaxScaler()
x_scaler = x_scaler.fit(custom_scale_matrix)
y_scaler = MinMaxScaler()
y_scaler = y_scaler.fit(custom_scale_matrix[:, 0:2])

def scale_data(training_data, training_labels, test_data, test_labels):

    norm_training_data = x_scaler.transform(training_data)
    norm_test_data = x_scaler.transform(test_data)
    norm_training_labels = y_scaler.transform(training_labels)
    norm_test_labels = y_scaler.transform(test_labels)
    
    
    return norm_training_data, norm_training_labels, norm_test_data, norm_test_labels
    

In [6]:
def batch_generator(batch_size, sequence_length):
    while True:
        x_shape = (batch_size, sequence_length, x_dim)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        y_shape = (batch_size, sequence_length, y_dim)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        for i in range(batch_size):
            rand_start = np.random.randint(len(norm_training_data) - sequence_length)
            
            x_batch[i] = norm_training_data[rand_start : rand_start + sequence_length]
            y_batch[i] = norm_training_labels[rand_start : rand_start + sequence_length]
            
        yield (x_batch, y_batch)

In [7]:
def custom_loss(y_true, y_pred):
    
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]
    
    
#     ts = tf.reshape(y_true_slice, (-1, 2))
    
#     ps = tf.reshape(y_pred_slice, (-1, 2))
    
#     def get_min_dist(pi):
# #         print(pi.get_shape(), ts.get_shape())
#         eu_dists = tf.norm(ts-pi, ord='euclidean', axis=1)
#         min_dist = tf.reduce_min(eu_dists)
#         return min_dist


#     min_dists = tf.map_fn(get_min_dist, ps, dtype=tf.float32)
#     print("Minimum distances to Ground Truth Points Shape", min_dists.get_shape())
#     mean_dist = tf.reduce_mean(min_dists)
#     print("Mean Mimimum Distance Shape", mean_dist.get_shape())
    
# #     min_dists_tot = 0.0
# #     print(ps.shape)
# #     for pi in ps:
# #         min_t = sys.float_info.max
# #         for ti in ts:
# #             t = tf.norm(ti-pi, ord='euclidean')
# #             if(t < min_t):
# #                 min_t = t
# #         min_dists_tot += min_t
        
# #     mean_dist = min_dists_tot/len(ps)
    
    eu_dists = tf.norm(y_true_slice-y_pred_slice, ord='euclidean')
    loss_mean = tf.reduce_mean(eu_dists)
    
    return loss_mean

In [8]:
class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure(figsize=(20, 10))
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.figure(figsize=(10, 8))
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.ioff()
        plt.show();

In [9]:
def plot_comparison(filename, start_index=0, individual_signals=False, train_percentage=0.8):
    training_data, training_labels, test_data, test_labels, orig_gps = load_train_test_data(filename, train_percentage)

    
    trial_input = np.concatenate((training_data, test_data))
    ground_truth = np.concatenate((training_labels, test_labels))

    ##Scale down the trial input, get predicted output
    trial_input = x_scaler.transform(trial_input)
    trial_input = np.expand_dims(trial_input, axis=0)
    predicted_output = model.predict(trial_input)
    
    
    ## Remove data upto the start point
    ground_truth = ground_truth[start_index:]
    orig_gps = orig_gps[start_index:]
    predicted_output = y_scaler.inverse_transform(predicted_output[0])[start_index:]
    

    # The output of the model is between 0 and 1. Inverse rescale to get input
    if(individual_signals):
        for signal in range(ground_truth_output.shape[1]):

            # Make the plotting-canvas bigger.
            plt.figure(figsize=(8,5))

            # Plot and compare the ground truth and Model predicted versions
            plt.plot(ground_truth[:, signal],'ro', label='true')
            plt.plot(predicted_output[:, signal], label='pred')

            # Plot grey box for warmup-period.
            p = plt.axvspan(0, warmup_steps, facecolor='red', alpha=0.15)

            # Plot labels etc.
            plt.ylabel("X Displacement" if signal == 0 else "Y Displacement")
            plt.xlabel("Timestep")
            plt.legend()
            plt.show()
            
    ##Print Graphs of X against Y
    plt.figure(figsize=(8,5))
    plt.plot(ground_truth[:, 0], ground_truth[:, 1], label='true', color = 'g')
    plt.plot(predicted_output[:len(training_data), 0], predicted_output[:len(training_data), 1], 
             label='predicted training', color = 'b')
    plt.plot(predicted_output[len(training_data):, 0], predicted_output[len(training_data):, 1], 
             label='predicted test', color = 'c')
    plt.plot(orig_gps[:, 1], orig_gps[:, 2], label='original gps', color = 'r')
    plt.legend()
    plt.show()
    
    print("Accuracy of GPS: ", measure_accuracy(ground_truth, orig_gps[:, 1:3]))
    print("Accuracy of RNN: ", measure_accuracy(ground_truth, predicted_output))
        

### Print Data with Pandas

In [10]:
# if(print_outputs):

#     if(False):
#         training_frame = pd.DataFrame(np.concatenate((training_data, training_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y', 
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     else:
#         training_frame = pd.DataFrame(np.concatenate((training_data, training_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y',
#     #                                                           'Vel X', 'Vel Y',
#     #                                                           'Mag X', 'Mag Y',
#     #                                                           'Gyro X', 'Gyro Y', 'Gyro Z',
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     print("Training Data")
#     display(training_frame.head(5))
#     training_frame.plot(figsize=(20,10))

In [11]:
# if(print_outputs):

#     if(False):
#         test_frame = pd.DataFrame(np.concatenate((test_data, test_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y', 
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     else:
#         test_frame = pd.DataFrame(np.concatenate((test_data, test_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y',
#     #                                                           'Vel X', 'Vel Y',
#     #                                                           'Mag X', 'Mag Y',
#     #                                                           'Gyro X', 'Gyro Y', 'Gyro Z',
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     print("Testing Data")
#     display(test_frame.head(5))
#     # test_frame.plot(figsize=(20,10))
